# Validating measures

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import time

### Data Directory

In [2]:
INPUT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/processed/'

### Reading predicted file

In [3]:
ip_file = INPUT_DIR + 'fd003_dbscan_results_4000_0.001.csv'
df_pred = pd.read_csv(ip_file, sep=',')

In [4]:
df_pred.head()

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas10,...,meas12,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels
0,-0.0005,0.0004,642.36,1583.23,1396.84,21.61,553.97,2387.96,9062.17,1.3,...,522.31,2388.01,8145.32,8.4246,391,39.11,23.3537,1,1,0
1,0.0008,-0.0003,642.50,1584.69,1396.89,21.61,554.55,2388.00,9061.78,1.3,...,522.42,2388.03,8152.85,8.4403,392,38.99,23.4491,1,2,0
2,-0.0014,-0.0002,642.18,1582.35,1405.61,21.61,554.43,2388.03,9070.23,1.3,...,522.03,2388.00,8150.17,8.3901,391,38.85,23.3669,1,3,0
3,-0.0020,0.0001,642.92,1585.61,1392.27,21.61,555.21,2388.00,9064.57,1.3,...,522.49,2388.08,8146.56,8.3878,392,38.96,23.2951,1,4,0
4,0.0016,0.0000,641.68,1588.63,1397.65,21.61,554.74,2388.04,9076.14,1.3,...,522.58,2388.03,8147.80,8.3869,392,39.14,23.4583,1,5,0


In [5]:
# Make everything in labels column into 0 or -1
df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)

### Generating Ground Truth with heuristics

In [6]:
timecycles_for_failure = 10

In [7]:
df_pred['ground_truth'] = 0

In [8]:
df_pred.head()

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas10,...,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth
0,-0.0005,0.0004,642.36,1583.23,1396.84,21.61,553.97,2387.96,9062.17,1.3,...,2388.01,8145.32,8.4246,391,39.11,23.3537,1,1,0,0
1,0.0008,-0.0003,642.50,1584.69,1396.89,21.61,554.55,2388.00,9061.78,1.3,...,2388.03,8152.85,8.4403,392,38.99,23.4491,1,2,0,0
2,-0.0014,-0.0002,642.18,1582.35,1405.61,21.61,554.43,2388.03,9070.23,1.3,...,2388.00,8150.17,8.3901,391,38.85,23.3669,1,3,0,0
3,-0.0020,0.0001,642.92,1585.61,1392.27,21.61,555.21,2388.00,9064.57,1.3,...,2388.08,8146.56,8.3878,392,38.96,23.2951,1,4,0,0
4,0.0016,0.0000,641.68,1588.63,1397.65,21.61,554.74,2388.04,9076.14,1.3,...,2388.03,8147.80,8.3869,392,39.14,23.4583,1,5,0,0


In [9]:
units = np.unique(df_pred['unit'])

In [10]:
df_pred.loc[df_pred['unit'] == -1]

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas10,...,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth


In [11]:
# Assign outliers for the last 'n' time cycles for each of the unit

units = np.unique(df_pred['unit'])
# print('Number of unique units:', len(units))
for current_unit in units:
    df_sub = df_pred.loc[df_pred['unit'] == current_unit]
    max_timecycle = max(df_sub['time_cycles'])    
    outlier_time_threshold = max_timecycle - timecycles_for_failure
    
    # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)
    
    df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1

/Users/talat/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
print(df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].loc[df_pred['unit'] == 1])

     unit  time_cycles  labels  ground_truth
0       1            1       0             0
1       1            2       0             0
2       1            3       0             0
3       1            4       0             0
4       1            5       0             0
5       1            6       0             0
6       1            7       0             0
7       1            8       0             0
8       1            9       0             0
9       1           10       0             0
10      1           11       0             0
11      1           12       0             0
12      1           13       0             0
13      1           14       0             0
14      1           15       0             0
15      1           16       0             0
16      1           17       0             0
17      1           18       0             0
18      1           19       0             0
19      1           20       0             0
20      1           21       0             0
21      1 

In [13]:
print('Number of outliers in the predicted values', len(df_pred.loc[df_pred['labels'] == -1]))
print('Number of outliers in the ground truth', len(df_pred.loc[df_pred['ground_truth'] == -1]))

Number of outliers in the predicted values 1030
Number of outliers in the ground truth 1000


In [14]:
# Copy into a new dataframe with only the required columns
df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()

In [15]:
# tp = 0
# fp = 1
# fn = 2
# tn = 3

def gen_vals(ground_truth, pred_label, pos_label, neg_label):
    if (ground_truth == pos_label) & (pred_label == pos_label):
        return 0
    elif (ground_truth == neg_label) & (pred_label == pos_label):
        return 1
    elif (ground_truth == pos_label) & (pred_label == neg_label):
        return 2
    elif (ground_truth == neg_label) & (pred_label == neg_label):
        return 3

### Find out the f-measure

In [16]:
pos_label = -1
neg_label = 0

df_scoring['pos'] = -1

df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)

In [17]:
from collections import Counter
counter = Counter(df_scoring['pos'])

print(counter)

accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
print('Accuracy = ', accuracy)

precision = counter[0] / (counter[0] + counter[1])
print('Precision = ', precision)

recall = counter[0] / (counter[0] + counter[2])
print('Recall = ', recall)

f1_score = 2 * (precision * recall) / (precision + recall)
print('F1 score: ', f1_score)

Counter({3: 23363, 0: 673, 1: 357, 2: 327})
Accuracy =  0.9723300970873786
Precision =  0.6533980582524271
Recall =  0.673
F1 score:  0.6630541871921183


# Making it into a function

In [18]:
def find_eval_metrics(df_pred, timecycles_for_failure):

    # Make everything in labels column into 0 or -1
    df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)
    
    df_pred['ground_truth'] = 0
    
    # Assign outliers for the last 'n' time cycles for each of the unit
    units = np.unique(df_pred['unit'])
    # print('Number of unique units:', len(units))
    for current_unit in units:
        df_sub = df_pred.loc[df_pred['unit'] == current_unit]
        max_timecycle = max(df_sub['time_cycles'])    
        outlier_time_threshold = max_timecycle - timecycles_for_failure

        # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)

        df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1
        
    # Copy into a new dataframe with only the required columns
    df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()
    
    pos_label = -1
    neg_label = 0

    df_scoring['pos'] = -1
    df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)
    
    counter = Counter(df_scoring['pos'])

    accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
    # print('Accuracy = ', accuracy)
    precision = counter[0] / (counter[0] + counter[1])
    # print('Precision = ', precision)
    recall = counter[0] / (counter[0] + counter[2])
    # print('Recall = ', recall)
    f1_score = 2 * (precision * recall) / (precision + recall)
    # print('F1 score: ', f1_score)
    
    return accuracy, precision, recall, f1_score

In [19]:
import warnings
warnings.filterwarnings("ignore")

max_timecycles_for_failure = 40

print('| timecycles_for_failure|', 'accuracy', '|', 'precision', '|', 'recall', '|', 'f1_score', '|')
print('|--|--|--|--|--|')

for i in range(1, max_timecycles_for_failure+1):
    accuracy, precision, recall, f1_score = find_eval_metrics(df_pred, i)
    print('|', i, '|', accuracy, '|', precision, '|', recall, '|', f1_score, '|')

| timecycles_for_failure| accuracy | precision | recall | f1_score |
|--|--|--|--|--|
| 1 | 0.9615695792880259 | 0.08737864077669903 | 0.9 | 0.15929203539823011 |
| 2 | 0.9646440129449838 | 0.17281553398058253 | 0.89 | 0.28943089430894314 |
| 3 | 0.9673139158576052 | 0.25339805825242717 | 0.87 | 0.39248120300751876 |
| 4 | 0.9694983818770226 | 0.32815533980582523 | 0.845 | 0.4727272727272727 |
| 5 | 0.9715210355987055 | 0.4009708737864078 | 0.826 | 0.5398692810457516 |
| 6 | 0.972411003236246 | 0.4601941747572815 | 0.79 | 0.5815950920245399 |
| 7 | 0.9727346278317152 | 0.512621359223301 | 0.7542857142857143 | 0.6104046242774567 |
| 8 | 0.9726537216828479 | 0.5601941747572815 | 0.72125 | 0.6306010928961747 |
| 9 | 0.9728155339805825 | 0.6106796116504855 | 0.6988888888888889 | 0.6518134715025906 |
| 10 | 0.9723300970873786 | 0.6533980582524271 | 0.673 | 0.6630541871921183 |
| 11 | 0.9713592233009709 | 0.6902912621359223 | 0.6463636363636364 | 0.6676056338028169 |
| 12 | 0.970307443365695